# Preprocessing for Nitrogen Loading Model (Nload)

## Objectives

Reclassify land use to including cranberry agriculture within subwatersheds of the Agawam (Wareham) drainage. Generate a table of area for 13 land use classes within each subcatchment, by sorting land uses from the following categories. 

Class Number|	Class Name
---|---
2|	Impervious
5|	Developed Open Space
6|	Cultivated Land
7|	Pasture/Hay
8|	Grassland
9|	Deciduous Forest
10|	Evergreen Forest
12|	Scrub/Shrub
13|	Palustrine Forested Wetland (C-CAP)
14|	Palustrine Scrub/Shrub Wetland (C-CAP)
15|	Palustrine Emergent Wetland (C-CAP)
16|	Estuarine Forested Wetland (C-CAP)
17|	Estuarine Scrub/Shrub Wetland (C-CAP)
18|	Estuarine Emergent Wetland (C-CAP)
19|	Unconsolidated Shore
20|	Bare Land
21|	Open Water
22|	Palustrine Aquatic Bed (C-CAP)
23|	Estuarine Aquatic Bed (C-CAP)


USEGENCODE|	USEGENNAME
---|---
0|	Unknown
2|	Open land
3|	Commercial
4|	Industrial
6|	Forest
7|	Agriculture
8|	Recreation
9|	Tax exempt
10| Mixed use, primarily residential
11|	Residential - single family
12|	Residential - multi-family
13|	Residential - other
20|	Mixed use, other
30|	Mixed use, primarily commercial
55|	Right-of-way
88|	Water


## References 
Kennedy, C. D., & Hoekstra, B. R. (2021). Measuring and modeling nitrogen export from cranberry farms. Ecosphere, 12(12), 1–15. https://doi.org/10.1002/ecs2.3686

Valiela, I., Geist, M., McClelland, J., & Tomasky, G. (2000). Nitrogen loading from watersheds to estuaries: Verification of the Waquoit Bay Nitrogen Loading Model. Biogeochemistry, 49(3), 277–293. https://doi.org/10.1023/A:1006345024374

Valiela, I., Collins, G., Kremer, J., Lajtha, K., Geist, M., Seely, B., … Sham, C. H. (1997). Nitrogen loading from coastal watersheds to receiving estuaries: New method and application. Ecological Applications, 7(2), 358–380. https://doi.org/10.1890/1051-0761(1997)007[0358:NLFCWT]2.0.CO;2

## Set up environment

In [1]:
# Load dependancies ***************************************
import arcpy as ap
import os, glob, re, sys
import pandas as pd
import numpy as np
#import scipy.stats

# Set up arcpy environment ******************************
# input data directory
idr = "C:\\Workspace\\Geodata\\Massachusetts\\"
# working directory this should be where arcgis project is located
wdr = "C:\\Workspace\\Geodata\\Nload\\"
# output directory
odr = os.path.join(wdr,'outputs')
# arcpy settings
ap.env.workspace = wdr # set arcpy environment working directory
aprx = arcpy.mp.ArcGISProject("Current") # current project
ap.CheckOutExtension("Spatial") # check out spatial extension
ap.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 UTM Zone 19N") # define cordinate system
# Allow overwriting of output
arcpy.env.overwriteOutput = True
#ap.SetProgressor("step", "processing...", step_value = 5)

Tip: see this [page](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/setprogressor.htm) for tips on arcpy settings. 

Tip: Use `os.path.join` to safely concatenate file paths. To do this enter folder names in quotes separated by commas.

Tip: Test in the geoprocessing tab first, then, once working, can copy python code from the geoprocessing tab when you click on the arrow next to `run`!

In [2]:
# check that directories exist
if os.path.exists(wdr) is False:
    sys.exit(wdr+" does not exist")
if os.path.exists(idr) is False:
    sys.exit(idr+" does not exist")
if os.path.exists(odr):
    print(odr+" already exists")
else:
    os.mkdir(odr)

C:\Workspace\Geodata\Nload\outputs already exists


Tip: Use `arcpy.FeatureSet` to add an input feature as an object to the current environment. Alternatively, `arcpy.management.MakeFeatureLayer` can be used to add input feature as a layer to the current project. The added layer should appear on the map. `arcpy.Copyfeatures_management` works in a similar way to `MakeFeatureLayer`. 

## Load input files

In [4]:
# Add Input Data *********************************** 
# Massachusetts 2016 Land Use Land Cover Layer 
path = os.path.join(idr,"lclu_gdb","MA_LCLU2016.gdb","LANDCOVER_LANDUSE_POLY")
MA_LCLU2016 = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_lclu16")

# Massachusetts cranberry agriculture (source: Jim McLaughlin)
# this needs to be updated
path = os.path.join(idr,'WMAbogsDRAFT2013\\WMAbogsDRAFT2013.shp')
os.path.exists(path)
arcpy.CopyFeatures_management(path,"in_bogs")
bogs = ap.FeatureSet(path)
#ap.management.MakeFeatureLayer(path, "lyr_ggbp")
path = os.path.join(idr,'Wareham_Bogs_v3\\WarehamBogs_master_v3.shp')
os.path.exists(path)
arcpy.CopyFeatures_management(path,"in_wareham_bogs")
wareham_bogs = ap.FeatureSet(path)

# Subcatchments of Agawam (Wareham) with terminus classifications
path = os.path.join(idr,"agawam_WS","agawam_subw_0309.shp")
agawam = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_agawam")

# Subcatchments of Great, Green, and Bourne Ponds with terminus classifications
path = os.path.join(idr,"GGBsheds","Great_Green_Bourne_Ponds.shp")
os.path.exists(path)
ggbp = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_ggbp")

<Result 'in_ggbp'>

## Main Program

### Part 1: Combine spatial layers

`arcpy.analysis.Intersect` (A) land use and (B) subwatershed Layer. 

In [5]:
# Intersect MA land use layer with subwatershed layer
inFeatures = [MA_LCLU2016, agawam]
intersectOutput = "out_agawam_lclu16"
arcpy.analysis.Intersect(inFeatures, intersectOutput, "", "", "INPUT")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_agawam_lclu16.shp'>

Spatial (A) join cranberry and (B) intersection of subwatershed land use layers. 

In [11]:
# change FID as BOG_FID in attribute table
arcpy.management.CalculateField("prj_wareham_bogs","BOG_FID","!FID!", 
                                "PYTHON3",'',"TEXT","NO_ENFORCE_DOMAINS")
                                
inFeat = intersectOutput
identFeat = "in_wareham_bogs"
identOutput = "out_lclu16_agawam_cran_ident"
arcpy.analysis.Identity(inFeat, identFeat, identOutput, "ALL", None, "NO_RELATIONSHIPS")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_lclu16_agawam_cran_ident.shp'>

<Result 'out_lclu16_agawam_cran_ident'>

Calculate area of polygons in hectares and export attribute tables to pandas dataframe

In [18]:

def fn_Calculate_Area_Ha(fc):
    # calculate area in hectares and add column to attribute table
    arcpy.management.CalculateGeometryAttributes(fc,
                                             "Area_Ha AREA_GEODESIC",
                                             '', "HECTARES", None, "SAME_AS_INPUT")

# path to feature class
fc1 = "in_agawam"
fn_Calculate_Area_Ha(fc1)
fc2 = identOutput
fn_Calculate_Area_Ha(fc2)

In [34]:
def fn_featureClass_Attributes_to_DataFrame(fc):
    # list of field names that excludes the shape geometry (including geometry messes this up)
    flds = [fld.name for fld in arcpy.ListFields(fc) if fld.name != arcpy.Describe(fc).shapeFieldName]
    arr = arcpy.da.FeatureClassToNumPyArray(fc,flds)
    df = pd.DataFrame(arr)
    return(df)

in_df1 = fn_featureClass_Attributes_to_DataFrame(fc1).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})
in_df2 = fn_featureClass_Attributes_to_DataFrame(fc2).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

In [35]:
# check that area of the inputs and outputs match closely
Total_Area1 = in_df1.Area_Ha.sum()
Total_Area2 = in_df2.Area_Ha.sum()
print(fc1+" Area (Ha):"+str(Total_Area1))
print(fc2+" Area (Ha):"+str(Total_Area2))
print("Difference in Area:"+str(Total_Area1 - Total_Area2)) # difference should be close to 0.0

in_agawam Area (Ha):12009.71311226636
out_lclu16_agawam_cran_ident Area (Ha):12007.009465318908
Difference in Area:2.703646947451489


In [36]:
# write attribute tables to csv
in_df1.to_csv(os.path.join(odr,fc1+".csv"))
in_df2.to_csv(os.path.join(odr,fc2+".csv"))

### Part 1: Reclassify combined layers for Nload Scenarios

Read excel tables with subwatershed land use area and terminus information

In [37]:
display(in_df1)
display(in_df2)

,FID,sf_area,SubWat_Num,New_ID,ft_perim,SubW_Names,SUB,Terminus,F_,Area_Ha
0,0,3.471427e+07,0,0,65465.877695,Estuary,0,e,0.0,322.510995
1,1,7.005651e+07,2,2,110369.417469,Agawam Reservoir S LT10,2,p,0.0,650.865795
2,2,2.110986e+06,19,19,8946.781816,Fearing Pond GT10,19,p,0.0,19.612364
3,3,1.227794e+07,21,21,17597.201223,Fearing Pond LT10,21,p,0.0,114.069335
4,4,6.956152e+06,24,24,17478.938892,Little Long Pond GT10,24,p,0.0,64.626888
5,5,1.361948e+07,25,25,22761.742210,Little Long Pond LT10,25,p,0.0,126.532533
6,6,4.162610e+06,22,22,9198.367018,Abner Pond,22,p,0.0,38.672989
7,7,6.418631e+06,23,23,13775.354863,Five Mile Pond,23,p,0.0,59.632582
8,8,2.098890e+07,26,26,32094.470678,East Branch N,26,s,0.0,194.998012
9,9,1.078122e+07,30,30,18422.485411,Charge Pond LT10,30,p,0.0,100.163457


,FID,FID_out_ag,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_agawam,sf_area,SubWat_Num,New_ID,ft_perim,SubW_Names,SUB,Terminus,F_,Area_Ha,FID_in_war,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,PERMIT_NUM,CropStatus,Bog_Code,Bog_Name_2,Active,Plant_Year,Variety,Var_Class,Bog_type,Area_sqft,Area_ha_1,renovated,ren_yr,new_upland,new_yr,New_ID_1,SubW_Nam_1,Basin_1,BOG_FID
0,0,0,15106589,Evergreen Forest,10,Tax exempt,9,9100,FEE,2014,52,R11C23,47538.459262,2.497232e+06,29,1.151071e+08,32,32,49039.314419,Wankinco R N LT10,32,s,0.0,2.892354,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,
1,1,1,15106589,Evergreen Forest,10,Tax exempt,9,9100,FEE,2014,52,R11C23,47538.459262,2.497232e+06,39,3.042836e+07,31,31,55143.984965,Wankinco R N GT10,31,s,0.0,16.626349,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,
2,2,2,15176018,Evergreen Forest,10,Residential - single family,11,101,FEE,2016,239,R11C24,17128.648040,2.212568e+05,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,13.167750,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,
3,3,3,15176200,Evergreen Forest,10,Recreation,8,802,FEE,2016,239,R11C24,378.716776,2.089846e+03,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,0.208984,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,
4,4,4,15176201,Evergreen Forest,10,Recreation,8,802,FEE,2016,239,R11C24,8.480932,3.279860e+00,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,0.000328,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114645,114645,112096,17383246,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2267.972452,5.345633e+04,45,7.421453e+07,42,42,57642.350438,Agawam River,42,s,0.0,0.000080,320,42431025,Tweedy & Barnes Co.,Charge Pond Rd.,Wareham,4,Sandusky Bog,Jim McLaughlin,BRP_WMA,2006-09-15,,42431025,,active,,,,,,,,92188.601562,0.856464,,,,,35,bog stream,Direct to Estuary,320
114646,114646,112933,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,33,1.681114e+07,35,35,24571.786710,bog stream,35,s,0.0,0.000039,323,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,179727.000000,1.669730,,,,,0,,Direct to Estuary,323
114647,114647,112934,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,45,7.421453e+07,42,42,57642.350438,Agawam River,42,s,0.0,0.000039,323,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,179727.000000,1.669730,,,,,0,,Direct to Estuary,323
114648,114648,112933,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,33,1.681114e+07,35,35,24571.786710,bog stream,35,s,0.0,0.000456,325,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,124196.000000,1.153820,yes,1995-2001,,,0,,Direct to Estuary,325


Estimate fraction of total population (k_Pop) living in each subwatershed using one of two methods:
1. Impervious Area (USEGENCODE == 2): dividing the total impervious area by the impervious area within each subwatershed. 
2. Residential Use Codes (USEGENCODE == 10,11,12,13): classify polygons as housing units if they fall under a residential use code

In [76]:
in_df2["HousingUnit"] = (in_df2.COVERCODE == 2) & np.isin(in_df2.USEGENCODE,[10,11,12,13])
totalHousingUnits = in_df2['HousingUnit'].sum()
df_HU_SUB = in_df2.groupby('SUB')['HousingUnit'].sum().reset_index(name='HousingUnits')
df_HU_SUB['Housing_Fraction'] = df_HU_SUB['HousingUnits']/totalHousingUnits
display(df_HU_SUB)

,SUB,HousingUnits,Housing_Fraction
0,0,22,0.001793
1,1,7,0.000571
2,10,16,0.001304
3,11,2,0.000163
4,12,37,0.003016
5,13,19,0.001549
6,14,2,0.000163
7,15,9,0.000734
8,16,18,0.001467
9,17,8,0.000652


In [79]:
totalImpervious = in_df2.loc[in_df2['COVERCODE']==2, 'Area_Ha'].sum()
df_imp_sub = in_df2.loc[in_df2['COVERCODE']==2].groupby('SUB')['Area_Ha'].sum().reset_index(name='Area_Ha')
df_imp_sub['Impervious_Fraction'] = df_imp_sub['Area_Ha']/totalImpervious

In [85]:
df1 = in_df1[['SUB','Terminus']]
df1['k_Pop'] = None
df1 = pd.merge(df1,df_imp_sub[['SUB','Impervious_Fraction']])
df1 = pd.merge(df1,df_HU_SUB[['SUB','Housing_Fraction']])

# how well do these metrics align
#import matplotlib.pyplot as plt
#df1.plot('Housing_Fraction','Impervious_Fraction','scatter')
#plt.show()

df1['k_Pop'] = df1.loc[:,'Housing_Fraction']
display(df1)

__main__:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SUB,Terminus,k_Pop,Impervious_Fraction,Housing_Fraction
0,0,e,0.001793,0.000710,0.001793
1,2,p,0.005787,0.014778,0.005787
2,19,p,0.000652,0.000113,0.000652
3,21,p,0.002690,0.004998,0.002690
4,24,p,0.000734,0.002901,0.000734
5,25,p,0.001956,0.001844,0.001956
6,22,p,0.000489,0.001951,0.000489
7,23,p,0.000000,0.000769,0.000000
8,26,s,0.000571,0.003931,0.000571
9,30,p,0.002282,0.005686,0.002282


Reclassify land use using Nload cover codes for default scenario

In [194]:
# GENERATE DICTIONARY OF CONDITIONS FOR RECLASSIFYING LAND USE
active_cranberry = '((np.isin(df.CropStatus,"active"))|((df.COVERCODE == 21) & (df.USEGENCODE==7))|((df.COVERCODE == 6) & (np.isin(df.USEGENCODE,[7,0])|np.isin(df.USE_CODE,["71","710","260","710A","0730","0761","0760","000","0000","7100","0710","0360"]))))'
flowthrough = '(np.isin(df.Bog_type,"flowthrough"))'
waterbody = '(np.isin(df.COVERCODE,[22,21]) & np.invert('+active_cranberry+'))'
# condition dictionary
BASELINE = {
    # key (LUC): value (condition string)
    # 1: "Natural Uplands"
    1: '(df.COVERCODE >= 8) & (df.COVERCODE <= 12)',
    # 2: "Mowed Areas (Lawns, Sports Fields, and Golf Courses)"
    2: 'df.COVERCODE == 5',
    # 3: "Agriculture, Non Cranberry"
    3: 'np.isin(df.COVERCODE,[6,7]) & np.invert('+active_cranberry+')',
    # 4: "Agriculture, Active Cranberry, Flowthrough"
    4: active_cranberry+' & '+flowthrough,
    # 5: "Agriculture, Active Cranberry, Non Flowthrough"
    5: active_cranberry+' & (np.invert('+flowthrough+'))',
    # 6: "Impervious, Roads"
    6: '(df.COVERCODE == 2) & (df.USEGENCODE == 55)',
    # 7: "Impervious, Non-Roads"
    7: '(df.COVERCODE == 2) & (df.USEGENCODE is not 55)',
    # 8: "Recieving Body (Estuary)"
    8: '(df.SUB == "0") & ((df.COVERCODE == 23)|'+waterbody+')', # note that SUB is a string
    # 9: "Freshwater Ponds and Lakes"
    9: '(df.SUB != "0") &' + waterbody,
    # 10: "Wetlands"
    10: '(df.COVERCODE >= 13) & (df.COVERCODE <= 18)',
    # 11: "Retired Cranberry, Flowthrough"
    11: 'np.isin(df.CropStatus,"abandoned") & '+flowthrough,
    # 12: "Retired Cranberry, Non flowthrough"
    12: 'np.isin(df.CropStatus,"abandoned") & np.invert('+flowthrough+')',
    # 13: "Other (Bare land or Shoreline)"
    13: 'np.isin(df.COVERCODE,[19,20])'}

# checking condtions 
df = pd.DataFrame({"COVERCODE":[21,21]*2,
                   "SUB":["0","1"]*2,
                   "CropStatus":["active"]*2 + ["abandoned"]*2,
                   "USEGENCODE":[0,0,0,0],
                   "USE_CODE":[0,0,0,0]})
df["8"]= eval(BASELINE[8])
df["9"]= eval(BASELINE[9])
print(df)

C:\Users\Adrian.Wiegman\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
C:\Users\Adrian.Wiegman\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


   COVERCODE SUB CropStatus  USEGENCODE  USE_CODE      8      9
0         21   0     active           0         0  False  False
1         21   1     active           0         0  False  False
2         21   0  abandoned           0         0   True  False
3         21   1  abandoned           0         0  False   True


In [174]:
df = in_df2
for i in list(BASELINE.keys()): 
    print("CHECKING "+str(i)+"....",eval(BASELINE[i]))

CHECKING 1.... 0          True
1          True
2          True
3          True
4          True
          ...  
114645    False
114646    False
114647    False
114648    False
114649    False
Name: COVERCODE, Length: 114650, dtype: bool
CHECKING 2.... 0         False
1         False
2         False
3         False
4         False
          ...  
114645    False
114646    False
114647    False
114648    False
114649    False
Name: COVERCODE, Length: 114650, dtype: bool
CHECKING 3.... 0         False
1         False
2         False
3         False
4         False
          ...  
114645    False
114646    False
114647    False
114648    False
114649    False
Length: 114650, dtype: bool
CHECKING 4.... 0         False
1         False
2         False
3         False
4         False
          ...  
114645    False
114646    False
114647    False
114648    False
114649    False
Length: 114650, dtype: bool
CHECKING 5.... 0         False
1         False
2         False
3         False
4         F

In [196]:
# declare land use class (LUC) reclasssification function
def fn_reclassify_LUC(df,dictionary):
    condlist = "["+", ".join(list(dictionary.values()))+"]"
    choicelist = list(dictionary.keys())
    #print(condlist)
    #print(choicelist)
    df["LUC"] =  np.select(eval(condlist),choicelist,13)
    return(df)

In [197]:
# execute reclassification subroutine
df2 = fn_reclassify_LUC(in_df2,BASELINE)
#df2["LUC"] =  np.where(((df2['LUC']==9) & (df2['SUB']==0)),8,df['LUC'])
# sum land area by subcatchment and LUC
df2_SUB_CC_Area = df2.groupby(['SUB','LUC'])['Area_Ha'].sum().reset_index(name='Area_Ha')
display(df2_SUB_CC_Area)

# check that area is within one tenth of a Ha of input
print('Difference in Total Area (Ha):',round(Total_Area2 - df2_SUB_CC_Area['Area_Ha'].sum(),1))

,SUB,LUC,Area_Ha
0,0,1,0.952013
1,0,2,0.079489
2,0,6,0.002072
3,0,7,0.436730
4,0,8,305.381342
...,...,...,...
354,9,2,1.513350
355,9,7,1.442824
356,9,9,7.570656
357,9,10,0.169447


Difference in Total Area (Ha): -0.0


Reclassify terminus and land use based on cranberry management scenarios

In [198]:
# List of subwatersheds with cranberry at terminus
SUB_LIST_1 = [4, 31, 33, 34, 35, 41]
# List of subwatersheds with flowthrough cranberry at terminus
SUB_LIST_4 = [4, 33, 34]
# List of cranberry bogs at terminus of subwatersheds
BOG_FID_LIST_1 = [307,301,318,295,294,326,310,242,241,259,269,155,156,148,195,194,193,192]

In [199]:
# Dictionary of Scenario Conditions
LUC_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: 'False',
    #1: "Restore Select Bogs to Wetlands"
    1: 'np.isin(df2[\'BOG_FID\'],BOG_FID_LIST_1)',
    #2: "Restore All Retired Bogs to Wetlands"
    2: 'df2.CropStatus is "abandoned"',
    #3: "Retire All Flowthrough Bogs"
    3: 'df2.Bog_type is "flowthrough"',
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: 'df2.Bog_type is "flowthrough"'
}
Terminus_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: 'False',
    #1: "Restore Select Bogs to Wetlands"
    1: 'np.isin(df1[\'SUB\'],SUB_LIST_1)',
    #2: "Restore All Retired Bogs to Wetlands"
    2: 'False',
    #3: "Retire All Flowthrough Bogs"
    3: 'False',
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: 'np.isin(df1[\'SUB\'],SUB_LIST_4)'
}
Terminus_scenarios.keys()
Terminus_scenarios.values()

dict_values(['False', "np.isin(df1['SUB'],SUB_LIST_1)", 'False', 'False', "np.isin(df1['SUB'],SUB_LIST_4)"])

In [200]:
# For each scenario reclassify land use, then sum up area by land use and subwatershed
# default value if scenario condition is true
LUC_iftrue = 10
Terminus_iftrue = "w"
scens = [str(i) for i in range(5)]
list(LUC_scenarios.keys())

[0, 1, 2, 3, 4]

In [201]:
# For each scenario reclassify land use,
out_df2 = df2
out_df1 = df1
# default LUC value if scenario condition is true
LUC_iftrue = 10
# default Terminus value
Terminus_iftrue = "w"
for i in list(LUC_scenarios.keys()):
    print (i)
    # reset dataframe
    df2_i = df2
    df1_i = df1

    # alternative land use for scenario 
    if i == 3: 
        LUC_iftrue = 11
        
    # alter LUC based on scenario conditions
    #np.where(logic,if_true, if_false)
    out_df2["LUC"+str(i)] = np.where(eval(LUC_scenarios[i]),LUC_iftrue,df2.LUC)
    out_df1["Terminus"+str(i)] = np.where(eval(Terminus_scenarios[i]),Terminus_iftrue,df1.Terminus)
display(out_df2)
display(out_df1)

0
1
2
3
4


,FID,FID_out_ag,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_agawam,sf_area,SubWat_Num,New_ID,ft_perim,SubW_Names,SUB,Terminus,F_,Area_Ha,FID_in_war,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,PERMIT_NUM,CropStatus,Bog_Code,Bog_Name_2,Active,Plant_Year,Variety,Var_Class,Bog_type,Area_sqft,Area_ha_1,renovated,ren_yr,new_upland,new_yr,New_ID_1,SubW_Nam_1,Basin_1,BOG_FID,HousingUnit,LUC,LUC0,LUC1,LUC2,LUC3,LUC4
0,0,0,15106589,Evergreen Forest,10,Tax exempt,9,9100,FEE,2014,52,R11C23,47538.459262,2.497232e+06,29,1.151071e+08,32,32,49039.314419,Wankinco R N LT10,32,s,0.0,2.892354,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,,False,1,1,1,1,1,1
1,1,1,15106589,Evergreen Forest,10,Tax exempt,9,9100,FEE,2014,52,R11C23,47538.459262,2.497232e+06,39,3.042836e+07,31,31,55143.984965,Wankinco R N GT10,31,s,0.0,16.626349,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,,False,1,1,1,1,1,1
2,2,2,15176018,Evergreen Forest,10,Residential - single family,11,101,FEE,2016,239,R11C24,17128.648040,2.212568e+05,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,13.167750,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,,False,1,1,1,1,1,1
3,3,3,15176200,Evergreen Forest,10,Recreation,8,802,FEE,2016,239,R11C24,378.716776,2.089846e+03,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,0.208984,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,,False,1,1,1,1,1,1
4,4,4,15176201,Evergreen Forest,10,Recreation,8,802,FEE,2016,239,R11C24,8.480932,3.279860e+00,49,7.115734e+07,51,50,48465.385958,Halfway Pond,51,p,0.0,0.000328,-1,,,,,0,,,,1899-12-30,,,,,,,,,,,,0.000000,0.000000,,,,,0,,,,False,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114645,114645,112096,17383246,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2267.972452,5.345633e+04,45,7.421453e+07,42,42,57642.350438,Agawam River,42,s,0.0,0.000080,320,42431025,Tweedy & Barnes Co.,Charge Pond Rd.,Wareham,4,Sandusky Bog,Jim McLaughlin,BRP_WMA,2006-09-15,,42431025,,active,,,,,,,,92188.601562,0.856464,,,,,35,bog stream,Direct to Estuary,320,False,5,5,5,5,5,5
114646,114646,112933,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,33,1.681114e+07,35,35,24571.786710,bog stream,35,s,0.0,0.000039,323,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,179727.000000,1.669730,,,,,0,,Direct to Estuary,323,False,5,5,5,5,5,5
114647,114647,112934,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,45,7.421453e+07,42,42,57642.350438,Agawam River,42,s,0.0,0.000039,323,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,179727.000000,1.669730,,,,,0,,Direct to Estuary,323,False,5,5,5,5,5,5
114648,114648,112933,22469580,Cultivated,6,Agriculture,7,7100,FEE,2016,310,R12C24,2027.751649,5.626106e+04,33,1.681114e+07,35,35,24571.786710,bog stream,35,s,0.0,0.000456,325,42431065,Sure-Cran Services Inc.,Charge Pond Rd.,Wareham,4,Parker Mills Bog,Jim McLaughlin,BRP_WMA,2012-12-07,credits added 9/29/05; xfer May 2010,42431032,,active,,,,,,,,124196.000000,1.153820,yes,1995-2001,,,0,,Direct to Estuary,325,False,5,5,5,5,5,5


,SUB,Terminus,k_Pop,Impervious_Fraction,Housing_Fraction,Terminus0,Terminus1,Terminus2,Terminus3,Terminus4
0,0,e,0.001793,0.000710,0.001793,e,e,e,e,e
1,2,p,0.005787,0.014778,0.005787,p,p,p,p,p
2,19,p,0.000652,0.000113,0.000652,p,p,p,p,p
3,21,p,0.002690,0.004998,0.002690,p,p,p,p,p
4,24,p,0.000734,0.002901,0.000734,p,p,p,p,p
5,25,p,0.001956,0.001844,0.001956,p,p,p,p,p
6,22,p,0.000489,0.001951,0.000489,p,p,p,p,p
7,23,p,0.000000,0.000769,0.000000,p,p,p,p,p
8,26,s,0.000571,0.003931,0.000571,s,s,s,s,s
9,30,p,0.002282,0.005686,0.002282,p,p,p,p,p


In [202]:
# then sum up area by land use and subwatershed    
for i in list(LUC_scenarios.keys()):
    print (i)   
    # Aggregate Data with Pandas .groupby()
    df2_i = df2.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')

    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC',
                               'Terminus'+str(i):'Terminus',
                            'Area_Ha'+str(i):'Area_Ha'})
    df2_i.to_csv(os.path.join(odr,"Wareham_LUC_Area"+str(i)+".csv"))
    display(df2_i)
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(df2['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"Wareham_Terminus_and_Pop"+str(i)+".csv"))

0


,SUB,LUC,Area_Ha
0,0,1,0.952013
1,0,2,0.079489
2,0,6,0.002072
3,0,7,0.436730
4,0,8,305.381342
...,...,...,...
354,9,2,1.513350
355,9,7,1.442824
356,9,9,7.570656
357,9,10,0.169447


Difference in Total Area (Ha): -0.0


,SUB,Terminus,k_Pop,Impervious_Fraction,Housing_Fraction
0,0,e,0.001793,0.000710,0.001793
1,2,p,0.005787,0.014778,0.005787
2,19,p,0.000652,0.000113,0.000652
3,21,p,0.002690,0.004998,0.002690
4,24,p,0.000734,0.002901,0.000734
5,25,p,0.001956,0.001844,0.001956
6,22,p,0.000489,0.001951,0.000489
7,23,p,0.000000,0.000769,0.000000
8,26,s,0.000571,0.003931,0.000571
9,30,p,0.002282,0.005686,0.002282


1


...

,SUB,Terminus,k_Pop,Impervious_Fraction,Housing_Fraction
0,0,e,0.001793,0.000710,0.001793
1,2,p,0.005787,0.014778,0.005787
2,19,p,0.000652,0.000113,0.000652
3,21,p,0.002690,0.004998,0.002690
4,24,p,0.000734,0.002901,0.000734
5,25,p,0.001956,0.001844,0.001956
6,22,p,0.000489,0.001951,0.000489
7,23,p,0.000000,0.000769,0.000000
8,26,s,0.000571,0.003931,0.000571
9,30,p,0.002282,0.005686,0.002282


4


,SUB,LUC,Area_Ha
0,0,1,0.952013
1,0,2,0.079489
2,0,6,0.002072
3,0,7,0.436730
4,0,8,305.381342
...,...,...,...
354,9,2,1.513350
355,9,7,1.442824
356,9,9,7.570656
357,9,10,0.169447


Difference in Total Area (Ha): -0.0


,SUB,Terminus,k_Pop,Impervious_Fraction,Housing_Fraction
0,0,e,0.001793,0.000710,0.001793
1,2,p,0.005787,0.014778,0.005787
2,19,p,0.000652,0.000113,0.000652
3,21,p,0.002690,0.004998,0.002690
4,24,p,0.000734,0.002901,0.000734
5,25,p,0.001956,0.001844,0.001956
6,22,p,0.000489,0.001951,0.000489
7,23,p,0.000000,0.000769,0.000000
8,26,s,0.000571,0.003931,0.000571
9,30,p,0.002282,0.005686,0.002282






















## Appendix